In [ ]:
# !pip install evaluate datasets py7zr rouge_score

In [ ]:
import torch
import matplotlib.pyplot as plt

# suppress warnings generated by your code:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
warnings.filterwarnings('ignore')

from tqdm.auto import tqdm

import json

import numpy as np

import pandas as pd

from datasets import load_dataset

from evaluate import load

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Trainer, TrainingArguments, pipeline, set_seed

from peft import LoraConfig, get_peft_model, TaskType, replace_lora_weights_loftq
from peft import prepare_model_for_kbit_training, PeftModel, PeftConfig

In [ ]:
# get model and tokenizer for summatization and dataset

In [ ]:
model_ckp = "google-t5/t5-small"
original_model = AutoModelForSeq2SeqLM.from_pretrained(model_ckp, torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained(model_ckp)

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

In [ ]:
# Check if CUDA is available and set the device accordingly
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

# Move the model to the appropriate device
original_model.to(device)

T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Drop

In [ ]:
from datetime import datetime
import os

now = datetime.today().isoformat()
print(now)

peft_model_path = f"./flan-t5-base-{now}"

2025-03-13T13:39:52.632269


In [ ]:
# get the samsum dataset for a chat summary evaluation

In [ ]:
dataset = load_dataset('Samsung/samsum')

README.md:   0%|          | 0.00/7.04k [00:00<?, ?B/s]

samsum.py:   0%|          | 0.00/3.36k [00:00<?, ?B/s]

The repository for Samsung/samsum contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/Samsung/samsum.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


corpus.7z:   0%|          | 0.00/2.94M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/14732 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/819 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/818 [00:00<?, ? examples/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 818
    })
})

In [ ]:
dataset['train']



Dataset({
    features: ['id', 'dialogue', 'summary'],
    num_rows: 14732
})

In [ ]:
import nltk
from nltk.tokenize import sent_tokenize

nltk.download("punkt_tab")

example = dataset['train'][100]['dialogue']
example_reference = dataset['train'][100]['summary']
print(example)
sent_tokenize(example)


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


Gabby: How is you? Settling into the new house OK?
Sandra: Good. The kids and the rest of the menagerie are doing fine. The dogs absolutely love the new garden. Plenty of room to dig and run around.
Gabby: What about the hubby?
Sandra: Well, apart from being his usual grumpy self I guess he's doing OK.
Gabby: :-D yeah sounds about right for Jim.
Sandra: He's a man of few words. No surprises there. Give him a backyard shed and that's the last you'll see of him for months.
Gabby: LOL that describes most men I know.
Sandra: Ain't that the truth! 
Gabby: Sure is. :-) My one might as well move into the garage. Always tinkering and building something in there.
Sandra: Ever wondered what he's doing in there?
Gabby: All the time. But he keeps the place locked.
Sandra: Prolly building a portable teleporter or something. ;-)
Gabby: Or a time machine... LOL
Sandra: Or a new greatly improved Rabbit :-P
Gabby: I wish... Lmfao!


['Gabby: How is you?',
 'Settling into the new house OK?',
 'Sandra: Good.',
 'The kids and the rest of the menagerie are doing fine.',
 'The dogs absolutely love the new garden.',
 'Plenty of room to dig and run around.',
 'Gabby: What about the hubby?',
 "Sandra: Well, apart from being his usual grumpy self I guess he's doing OK.\r\nGabby: :-D yeah sounds about right for Jim.",
 "Sandra: He's a man of few words.",
 'No surprises there.',
 "Give him a backyard shed and that's the last you'll see of him for months.",
 'Gabby: LOL that describes most men I know.',
 "Sandra: Ain't that the truth!",
 'Gabby: Sure is.',
 ':-) My one might as well move into the garage.',
 'Always tinkering and building something in there.',
 "Sandra: Ever wondered what he's doing in there?",
 'Gabby: All the time.',
 'But he keeps the place locked.',
 'Sandra: Prolly building a portable teleporter or something.',
 ';-)\r\nGabby: Or a time machine... LOL\r\nSandra: Or a new greatly improved Rabbit :-P\r\nGab

In [ ]:
# evaluate baseline function
from datasets import DatasetDict

def baseline_summarize(dialoge):
  # The baseline summarization returns the first 3 lines
  return "\n".join(sent_tokenize(dialoge)[:3])

baseline_summarize(example)

'Gabby: How is you?\nSettling into the new house OK?\nSandra: Good.'

In [ ]:
pipe_t5_original = pipeline("text2text-generation", model=original_model, tokenizer=tokenizer, device=0, framework="pt")

Device set to use cuda:0


In [ ]:
sample_example = "summarize: \n" + example
print(sample_example,"\n")

summarize: 
Gabby: How is you? Settling into the new house OK?
Sandra: Good. The kids and the rest of the menagerie are doing fine. The dogs absolutely love the new garden. Plenty of room to dig and run around.
Gabby: What about the hubby?
Sandra: Well, apart from being his usual grumpy self I guess he's doing OK.
Gabby: :-D yeah sounds about right for Jim.
Sandra: He's a man of few words. No surprises there. Give him a backyard shed and that's the last you'll see of him for months.
Gabby: LOL that describes most men I know.
Sandra: Ain't that the truth! 
Gabby: Sure is. :-) My one might as well move into the garage. Always tinkering and building something in there.
Sandra: Ever wondered what he's doing in there?
Gabby: All the time. But he keeps the place locked.
Sandra: Prolly building a portable teleporter or something. ;-)
Gabby: Or a time machine... LOL
Sandra: Or a new greatly improved Rabbit :-P
Gabby: I wish... Lmfao! 



In [ ]:
pipe_out = pipe_t5_original(sample_example, batch_size=1, max_length=512, truncation=True)
pipe_out

[{'generated_text': 'Gabby: \'i wish... Lmfao!\' Sandra: "i wish... Lmfao!\''}]

In [ ]:
pipe_out[0]['generated_text']

'Gabby: \'i wish... Lmfao!\' Sandra: "i wish... Lmfao!\''

In [ ]:
# If I wanted to do the same using: tokenizer(), model.generate(), tokenizer.decode()

input_example = tokenizer(sample_example, truncation=True, max_length=1024, return_tensors="pt")
print(input_example, "\n")
output_model = original_model.generate(input_ids=input_example["input_ids"].to(device),
                                       attention_mask = input_example["attention_mask"].to(device),
                                       max_length = 64, num_beams=2, do_sample=True, top_k=50, length_penalty = 0.8)
print(output_model, "\n")
print(tokenizer.decode(output_model[0],skip_special_tokens=True))

{'input_ids': tensor([[21603,    10, 15894,   969,    10,   571,    19,    25,    58,  2821,
            17,   697,   139,     8,   126,   629,  6902,    58, 24120,    10,
          1804,     5,    37,  1082,    11,     8,   880,    13,     8,  1076,
           545,  1753,    33,   692,  1399,     5,    37,  3887,  2997,   333,
             8,   126,  2004,     5, 28474,    13,   562,    12,  9799,    11,
           661,   300,     5, 15894,   969,    10,   363,    81,     8,  7801,
           969,    58, 24120,    10,  1548,     6,  3943,    45,   271,   112,
          4680,     3,  9667,  1167,    63,  1044,    27,  3382,     3,    88,
            31,     7,   692,  6902,     5, 15894,   969,    10,     3,    10,
            18,   308, 17945,  2993,    81,   269,    21,  6006,     5, 24120,
            10,   216,    31,     7,     3,     9,   388,    13,   360,  1234,
             5,   465, 17844,   132,     5,  6434,   376,     3,     9, 11278,
         12478,    11,    24,    31,  

In [ ]:
# define aux generate function

def aux_batch(dataset, batch_size = 64):
  for i in range(1, len(dataset), batch_size):
    batch = dataset[i:i + batch_size]
    yield batch

In [ ]:
class CustomPipeline():
  def __init__(self, tokenizer=None, model=None,
               max_gen_length=64, max_inp_length=512,
               num_beams=2, do_sample=True, top_k=20,
               length_penalty=0.8, batched=False
               ):

    self.tokenizer = tokenizer
    self.model = model
    self.max_gen_length = max_gen_length
    self.max_inp_length = max_inp_length
    self.num_beams = num_beams
    self.do_sample = do_sample
    self.top_k = top_k
    self.length_penalty = length_penalty
    self.batched = batched

  def gen(self, input):

    model_ = self.model

    if self.batched:
      batch_output = []

      for batch in input:

        input_example = self.tokenizer(batch, truncation=True, max_length=self.max_inp_length, return_tensors="pt")

        output_model = model_.generate(input_ids=input_example["input_ids"].to(device),
                                       attention_mask = input_example["attention_mask"].to(device),
                                       max_length = self.max_gen_length, num_beams=self.num_beams, top_k = self.top_k,
                                         do_sample=self.do_sample, length_penalty = self.length_penalty)
        output_ = tokenizer.decode(output_model[0],skip_special_tokens=True)
        batch_output.append(output_)

      return batch_output

    else:

      input_example = self.tokenizer(input, truncation=True, max_length=self.max_inp_length, return_tensors="pt")

      output_model = model_.generate(input_ids=input_example["input_ids"].to(device),
                                       attention_mask = input_example["attention_mask"].to(device),
                                       max_length = self.max_gen_length, num_beams=self.num_beams, top_k = self.top_k,
                                         do_sample=self.do_sample, length_penalty = self.length_penalty)
      output = tokenizer.decode(output_model[0],skip_special_tokens=True)

      return output

In [ ]:
my_pipe = CustomPipeline(tokenizer=tokenizer, model=original_model,
                         max_gen_length = 64, max_inp_length = 512, num_beams=2, do_sample=False,
                         top_k = 50, batched = False, length_penalty=0.8)

out1 = my_pipe.gen(sample_example)

print(out1)

Sandra: Gabby is a man of few words. No surprises there. Sandra: Gabby keeps the place locked. Sandra: I wish... Lmfao!


In [ ]:
metric = load("rouge")

In [ ]:
def eval_baseline(dataset, batch_size=64, metric=metric,
                  target_column='dialogue', reference_column='summary'):

  num_iter = int(len(dataset) / batch_size)

  # Create a progress bar to track the training progress
  progress_bar = tqdm(range(num_iter))

  batch = aux_batch(dataset, batch_size=batch_size)
  for i in range(num_iter):
    batch_i = next(batch)
    batch_summary = [baseline_summarize(j) for j in batch_i[target_column]]

    metric.add_batch(predictions=batch_summary,
                 references=batch_i[reference_column])
    progress_bar.update(1)
  result = metric.compute()
  return result


In [ ]:
def eval_t5(dataset=None, batch_size=64, metric=metric, custom_pipeline=None,
            target_column='dialogue', reference_column='summary'
            ):
  num_iter = int(len(dataset) / batch_size)

  # Create a progress bar to track the training progress
  progress_bar = tqdm(range(num_iter))

  batch = aux_batch(dataset, batch_size=batch_size)
  for i in range(num_iter):
    batch_ = next(batch)
    batch_x = [ "summarize: " + j  for j in batch_[target_column]] # prepend "summarize: " to each element of batch
    batch_x = custom_pipeline.gen(batch_x) # returns a list [[list batch],...]
    #batch_x = pipeline(batch_x, batch_size=batch_size, max_length=512, truncation=True) # pipe out is a list of dictionaries [{'generated_text': str(summarized text)},...]
    #batch_x = [k['generated_text'] for k in batch_x] # transform the list of dictionaries in a list of str.

    metric.add_batch(predictions=batch_x,
                 references=batch_[reference_column])
    progress_bar.update(1)
  result = metric.compute()
  return result

In [ ]:
metric = load("rouge")
baseline_metrics = eval_baseline(dataset["test"], batch_size=64,
                                 metric=metric, target_column='dialogue', reference_column='summary')

  0%|          | 0/12 [00:00<?, ?it/s]

In [ ]:
baseline_metrics

{'rouge1': 0.2971433400781411,
 'rouge2': 0.08583223952454339,
 'rougeL': 0.22880676464482042,
 'rougeLsum': 0.2667197612749733}

In [ ]:
baseline_df = pd.DataFrame.from_dict(baseline_metrics, orient='index',columns=['Baseline metric'])
baseline_df

,Baseline metric
rouge1,0.297143
rouge2,0.085832
rougeL,0.228807
rougeLsum,0.266720


In [ ]:
custom_eval_base_pipeline = CustomPipeline(tokenizer=tokenizer, model=original_model,
                                max_gen_length = 64, max_inp_length = 512, num_beams=2, do_sample=True,
                                top_k = 50, batched = True, length_penalty=0.8
                                           )

t5_base_metrics = eval_t5(dataset=dataset["test"], batch_size=64, metric=metric,
                          custom_pipeline = custom_eval_base_pipeline,
                          target_column='dialogue', reference_column='summary')

  0%|          | 0/12 [00:00<?, ?it/s]

In [ ]:
t5_base_metrics

{'rouge1': 0.24614721858322153,
 'rouge2': 0.06374043323123035,
 'rougeL': 0.1937146856331064,
 'rougeLsum': 0.193680139330901}

In [ ]:
base_t5_df = pd.DataFrame.from_dict(t5_base_metrics, orient='index',columns=['Base T5-small'])
base_t5_df

,Base T5-small
rouge1,0.246147
rouge2,0.063740
rougeL,0.193715
rougeLsum,0.193680


## Now performing PEFT on the base model using the train set

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_ckp, torch_dtype=torch.bfloat16).to(device)

In [ ]:
def convert_examples2features(batched, tokenizer=tokenizer, target_column='dialogue', reference_column='summary'):

  input_encodings = tokenizer(batched[target_column], padding=True, max_length=512, return_tensors="pt")

  with tokenizer.as_target_tokenizer():
    target_encodings = tokenizer(batched[reference_column], padding=True, max_length=128, return_tensors="pt")

  return {'input_ids': input_encodings['input_ids'],
          'attention_mask': input_encodings['attention_mask'],
          'labels': target_encodings['input_ids']
          }


In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 818
    })
})

In [ ]:
preproc_ds = dataset.map(convert_examples2features, batched = True)

Map:   0%|          | 0/14732 [00:00<?, ? examples/s]

Map:   0%|          | 0/819 [00:00<?, ? examples/s]

Map:   0%|          | 0/818 [00:00<?, ? examples/s]

In [ ]:
preproc_ds.set_format(type='torch',columns=['input_ids', 'attention_mask', 'labels'])

In [ ]:
preproc_ds

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 818
    })
})

In [ ]:
lora_config = LoraConfig(
    r = 8, lora_alpha = 8,
    target_modules = ["q", "v"],
    lora_dropout = 0.05,
    bias = "none",
    task_type = TaskType.SEQ_2_SEQ_LM
)

In [ ]:
# Add LoRA adapter layers to the model
peft_model = get_peft_model(model, lora_config)

In [ ]:
peft_model

PeftModelForSeq2SeqLM(
  (base_model): LoraModel(
    (model): T5ForConditionalGeneration(
      (shared): Embedding(32128, 512)
      (encoder): T5Stack(
        (embed_tokens): Embedding(32128, 512)
        (block): ModuleList(
          (0): T5Block(
            (layer): ModuleList(
              (0): T5LayerSelfAttention(
                (SelfAttention): T5Attention(
                  (q): lora.Linear(
                    (base_layer): Linear(in_features=512, out_features=512, bias=False)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.05, inplace=False)
                    )
                    (lora_A): ModuleDict(
                      (default): Linear(in_features=512, out_features=8, bias=False)
                    )
                    (lora_B): ModuleDict(
                      (default): Linear(in_features=8, out_features=512, bias=False)
                    )
                    (lora_embedding_A): ParameterDict()
              

In [ ]:
def num_trainable_model_params(model1):
  return sum(p.numel() for p in model1.parameters() if p.requires_grad)

In [ ]:
print(num_trainable_model_params(peft_model))

294912


In [ ]:
print(num_trainable_model_params(original_model))

60506624


In [ ]:
output_dir = peft_model_path

peft_training_args = TrainingArguments(
    output_dir = output_dir,
    per_device_train_batch_size = 1,
    per_device_eval_batch_size = 1,
    learning_rate = 1e-3,
    num_train_epochs = 3,
    save_strategy="epoch",
    fp16=True,
    gradient_accumulation_steps = 16,
    evaluation_strategy = 'steps'
)

In [ ]:
from transformers import DataCollatorForSeq2Seq

seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, peft_model, return_tensors = "pt")

In [ ]:
peft_trainer = Trainer(
    model = peft_model,
    args = peft_training_args,
    data_collator = seq2seq_data_collator,
    train_dataset = preproc_ds['train'],
    eval_dataset = preproc_ds['validation']
)

In [ ]:
peft_trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: b-nunes-bruno (b-nunes-bruno-universidad-aut-noma-de-madrid) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss,Validation Loss
500,1.404300,0.646129
1000,0.706100,0.628923
1500,0.689200,0.613579
2000,0.674600,0.610414
2500,0.667400,0.605079


TrainOutput(global_step=2760, training_loss=0.8135434800300045, metrics={'train_runtime': 4040.737, 'train_samples_per_second': 10.938, 'train_steps_per_second': 0.683, 'total_flos': 9614766775468032.0, 'train_loss': 0.8135434800300045, 'epoch': 2.9970133043714364})

In [ ]:
peft_trainer.model.save_pretrained(peft_model_path)

In [ ]:
# Prepare the model for inference by adding on the adapters to original T5 small
peft_model_path = "/content/drive/MyDrive/flan-t5-base-2025-03-13T13:39:52.632269"

peft_model_base =AutoModelForSeq2SeqLM.from_pretrained(
    model_ckp,
    torch_dtype = torch.bfloat16
).to(device)

peft_inference_model = PeftModel.from_pretrained(
    peft_model_base,
    peft_model_path,
    torch_dtype = torch.bfloat16,
    is_trainable = False
).to(device)

In [ ]:
custom_eval_peft_pipeline = CustomPipeline(tokenizer=tokenizer, model=peft_inference_model,
                                max_gen_length = 64, max_inp_length = 512, num_beams=2, do_sample=True,
                                top_k = 50, batched = True, length_penalty=0.8
                                           )

t5_peft_metrics = eval_t5(dataset["test"], batch_size=64, metric=metric,
                          custom_pipeline = custom_eval_peft_pipeline,
                          target_column='dialogue', reference_column='summary')

  0%|          | 0/12 [00:00<?, ?it/s]

In [ ]:
t5_peft_metrics

{'rouge1': 0.3917747985422132,
 'rouge2': 0.16318316033920935,
 'rougeL': 0.32530587646645887,
 'rougeLsum': 0.32536125830891865}

In [ ]:
custom_peft_pipeline = CustomPipeline(tokenizer=tokenizer, model=peft_inference_model,
                                max_gen_length = 64, max_inp_length = 512, num_beams=2, do_sample=True,
                                top_k = 50, batched = False, length_penalty=0.8
                                           )

out2 = custom_peft_pipeline.gen(input = sample_example)


In [ ]:
print("Example dialogue:\n",(example),"\n")

print("Summary with base model t5-small:\n", out1,"\n")
print("Summary with PEFT model t5-small:\n", out2,"\n")


Example dialogue:
 Gabby: How is you? Settling into the new house OK?
Sandra: Good. The kids and the rest of the menagerie are doing fine. The dogs absolutely love the new garden. Plenty of room to dig and run around.
Gabby: What about the hubby?
Sandra: Well, apart from being his usual grumpy self I guess he's doing OK.
Gabby: :-D yeah sounds about right for Jim.
Sandra: He's a man of few words. No surprises there. Give him a backyard shed and that's the last you'll see of him for months.
Gabby: LOL that describes most men I know.
Sandra: Ain't that the truth! 
Gabby: Sure is. :-) My one might as well move into the garage. Always tinkering and building something in there.
Sandra: Ever wondered what he's doing in there?
Gabby: All the time. But he keeps the place locked.
Sandra: Prolly building a portable teleporter or something. ;-)
Gabby: Or a time machine... LOL
Sandra: Or a new greatly improved Rabbit :-P
Gabby: I wish... Lmfao! 

Summary with base model t5-small:
 Sandra: Gabby is

In [ ]:
peft_t5_df = pd.DataFrame.from_dict(t5_peft_metrics,orient='index',columns=['PEFT T5-small'])
peft_t5_df

,PEFT T5-small
rouge1,0.391775
rouge2,0.163183
rougeL,0.325306
rougeLsum,0.325361


In [ ]:
peft_t5_df.join(base_t5_df).join(baseline_df).T

,rouge1,rouge2,rougeL,rougeLsum
PEFT T5-small,0.391775,0.163183,0.325306,0.325361
Base T5-small,0.246147,0.063740,0.193715,0.193680
Baseline metric,0.297143,0.085832,0.228807,0.266720
